# Imports

This cell contains all the required **libraries and dependencies**
needed for preprocessing, tokenization, and further analysis.

In [1]:
import os
import pandas as pd
import re
import regex

from concurrent.futures import ThreadPoolExecutor
from conllu import parse
from difflib import SequenceMatcher
from pathlib import Path


# General Functions

This cell defines the **helper functions** used throughout the notebook.
They provide reusable functionality to support preprocessing, tokenization,
and other tasks required in later steps.

In [2]:
def normalize_quotes(text):
    replacements = {
        '"': '"',
        '“': '"',
        '”': '"',
        '‘': "'",
        '’': "'",
        "‘’": '"',
        "’‘": '"',
        "`": "'",
        "``": '"',
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text


def is_punctuation(text):
    return all(not c.isalnum() for c in text)


def is_number(word: str) -> bool:
    return bool(re.fullmatch(r'\d+', word))


def clean_word(word: str) -> str:
    return regex.sub(r'[^\p{L}]', '', word)


def clean_number(word: str) -> str:
    return re.sub(r'[^0-9]', '', word)

In [ ]:
def count_sentences_pos(conllu_dir):
    def count_sentences_in_file(file_path):
        try:
            with Path(file_path).open(encoding="utf-8") as f:
                data = f.read()

            sentences = parse(data)
            return len(sentences)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            return 0

    total_sentences = 0

    for i in range(1, 10):
        subdir = conllu_dir / f"{i}Part"
        if subdir.exists() and subdir.is_dir():
            for conllu_file in subdir.glob("*.conllu"):
                count = count_sentences_in_file(conllu_file)
                total_sentences += count

    return total_sentences


def count_sentences_ner(file_path):
    try:
        with Path(file_path).open(encoding="utf-8") as f:
            lines = f.readlines()

        sentence_count = sum(1 for line in lines if line.strip() == "")
        return sentence_count
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return 0

In [ ]:
def read_ner_data_and_assign_ids(file_path):
    try:
        with Path(file_path).open(encoding="utf-8") as f:
            lines = f.readlines()

        sentences = []
        current_sentence = []
        sentence_id = 1

        for line in lines:
            line = line.strip()
            if line == "":
                if current_sentence:
                    sentences.append((sentence_id, current_sentence))
                    current_sentence = []
                    sentence_id += 1
            else:
                parts = line.split()
                if len(parts) >= 2:
                    token, label = parts[0], parts[-1]
                    current_sentence.append((token, label))

        if current_sentence:
            sentences.append((sentence_id, current_sentence))

        return sentences
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return []


def read_pos_data_and_assign_ids(conllu_dir):
    sentences = []
    sentence_id = 1

    for i in range(1, 10):
        subdir = conllu_dir / f"{i}Part"
        if subdir.exists() and subdir.is_dir():
            for conllu_file in subdir.glob("*.conllu"):
                try:
                    with conllu_file.open(encoding="utf-8") as f:
                        data = f.read()

                    parsed_sentences = parse(data)
                    for sent in parsed_sentences:
                        tokens = [

                            (normalize_quotes(token['form']),
                             normalize_quotes(token['lemma']),
                             token['upostag'],
                             token['feats'],
                             token['head'],
                             token['deprel'],
                             token['deps'],
                             token['misc'])

                            for token in sent
                        ]
                        sentences.append((sentence_id, tokens))
                        sentence_id += 1
                except Exception as e:
                    print(f"Error processing {conllu_file}: {e}")

    return sentences

In [ ]:
ner_path = Path("../../Corpus/korpusi.txt")
pos_path = Path("../../Conllu Files in Corpus/")

In [ ]:
ner_sentences_count = count_sentences_ner(ner_path)
pos_sentences_count = count_sentences_pos(pos_path)

In [ ]:
print(f"NER sentences: {ner_sentences_count}")
print(f"POS sentences: {pos_sentences_count}")

In [ ]:
ner_sentences = read_ner_data_and_assign_ids(ner_path)
print(ner_sentences[0])

In [ ]:
pos_sentences = read_pos_data_and_assign_ids(pos_path)
print(pos_sentences[0])

# Initial Tokenization

This cell performs **tokenization** on each file.
All files are processed, even if their tokens do not align perfectly.
These results will later be leveraged for an additional round of tokenization.

In [ ]:
def read_ner_corpus(path):
    ner_words = []

    with Path(path).open(encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = [part.strip() for part in line.split("\t") if part.strip()]
            if len(parts) >= 2:

                clean_word = parts[0].replace(" ", "")
                if clean_word:  # Only add non-empty words
                    ner_words.append([clean_word, parts[1]])  # [word, tag]

    return ner_words

def read_text_files_and_match(text_dir: Path, ner_words, output_dir: Path):
    remaining_corpus = ner_words.copy()
    output_dir.mkdir(parents=True, exist_ok=True)
    pos_index = 0
    subdir = text_dir / f"7Part"
    if subdir.exists() and subdir.is_dir():
        files = sorted(subdir.glob("*.txt"), key=lambda f: int(f.stem.split("_")[0]))

        for txt_file in files:
            print(f"\nProcessing {txt_file} ...")

            try:
                with txt_file.open(encoding="utf-8") as f:
                    raw_sentences = [line.strip() for line in f if line.strip()]

                matched_lines = []

                for raw in raw_sentences:
                    space_tokens = [token.strip() for token in raw.split() if token.strip()]
                    if not space_tokens:
                        continue

                    all_tokens = []
                    for token in space_tokens:
                        current_token = ""
                        for char in token:
                            if char.isalnum() or char == '-':
                                current_token += char
                            else:
                                if current_token:
                                    all_tokens.append(current_token)
                                    current_token = ""
                                all_tokens.append(char)
                        if current_token:
                            all_tokens.append(current_token)

                    sentence_tokens = []

                    for token in all_tokens:
                        if pos_index < len(remaining_corpus):
                            corpus_word, tag = remaining_corpus[pos_index]

                            if token.lower() == corpus_word.lower():
                                sentence_tokens.append(f"{token}\t{tag}")
                                pos_index += 1
                            else:
                                sentence_tokens.append(f"{token}\tO")
                                print(f"Mismatch: token '{token}' vs corpus '{corpus_word}' at position {pos_index}")
                        else:
                            sentence_tokens.append(f"{token}\tO")

                    matched_lines.extend(sentence_tokens)
                    matched_lines.append("")

                out_subdir = output_dir / f"7Part2"
                out_subdir.mkdir(parents=True, exist_ok=True)
                out_file = out_subdir / txt_file.name
                with out_file.open("w", encoding="utf-8") as f:
                    f.write("\n".join(matched_lines))

                print(f" -> Wrote {len(matched_lines)} lines to {out_file}")

            except Exception as e:
                print(f"Error processing {txt_file}: {e}")

    remaining_corpus = remaining_corpus[pos_index:]
    return remaining_corpus

In [ ]:
ner_path = Path("../../Corpus/korpusi.txt")
text_dir = Path("../../Text Files in Corpus/")
output_dir = Path("../../Matched Text Files/")

ner_words = read_ner_corpus(ner_path)
remaining = read_text_files_and_match(text_dir, ner_words, output_dir)

print(f"\nDone. Remaining corpus tokens: {len(remaining)}")

# Final Tokenization

This cell performs the **final round of tokenization**.
Each token extracted from the corpus is written to its respective file.
This ensures a complete and organized representation of the data.

In [ ]:
def get_corpus_data(path):
    ner_words = []
    with Path(path).open(encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            line = line.replace("\t\t'", "\t")
            parts = [normalize_quotes(part.strip()) for part in line.split("\t") if part.strip()]
            if len(parts) >= 2:
                ner_words.append(parts[:2])
    return ner_words

def fix_the_ner_data_per_file(text_dir: Path, corpus_tokens, output_dir: Path):
    output_dir.mkdir(parents=True, exist_ok=True)
    corpus_index = 0  # track position in corpus

    for i in range(1, 11):
        subdir = text_dir / f"{i}Part"
        if not subdir.exists() or not subdir.is_dir():
            continue

        files = sorted(subdir.glob("*.txt"), key=lambda f: int(f.stem.split("_")[0]))
        for file in files:
            fixed_lines = []
            recon_word = ""

            file_ner_words = get_corpus_data(file)

            for ner_word_info in file_ner_words:
                ner_word = normalize_quotes(ner_word_info[0])

                if corpus_index >= len(corpus_tokens):
                    break

                corpus_word, corpus_tag = corpus_tokens[corpus_index]
                corpus_word = normalize_quotes(corpus_word)

                if ner_word.lower() == corpus_word.lower():
                    fixed_lines.append(f"{corpus_word}\t\t{corpus_tag}")
                    corpus_index += 1
                    recon_word = ""
                    continue

                recon_word += ner_word
                if recon_word.lower() == corpus_word.lower():
                    fixed_lines.append(f"{corpus_word}\t\t{corpus_tag}")
                    corpus_index += 1
                    recon_word = ""
                elif not corpus_word.startswith(recon_word):
                    recon_word = ""

            if len(fixed_lines) == 0:
                if corpus_index < len(corpus_tokens):
                    corpus_word, corpus_tag = corpus_tokens[corpus_index]
                    print(f"STOPPED: No matches found in file {file.name}. Next corpus word: {corpus_word}")
                else:
                    print(f"STOPPED: No matches found in file {file.name}. Corpus exhausted.")
                return

            out_subdir = output_dir / f"{i}Part"
            out_subdir.mkdir(parents=True, exist_ok=True)
            out_file = out_subdir / file.name
            with out_file.open("w", encoding="utf-8") as f:
                f.write("\n".join(fixed_lines))

            print(f"Wrote {len(fixed_lines)} lines to {out_file}")

    print(f"Done. Remaining corpus tokens: {len(corpus_tokens) - corpus_index}")


In [ ]:
ner_path = Path("../../Corpus/korpusi.txt")
text_dir = Path("../../Matched Text Files2/")
output_dir = Path("../../Replaced Text Files/")

In [ ]:
ner_words = get_corpus_data(ner_path)
fix_the_ner_data_per_file(text_dir, ner_words, output_dir)

# Dataset Matching

This cell handles the **alignment and matching** of datasets.
It ensures that the data sources are properly synchronized
before moving on to tokenization and further processing.

In [3]:
def fuzzy_ratio(a, b):
    """Return similarity ratio between two strings."""
    return SequenceMatcher(None, a, b).ratio()

def align_ner_pos(ner_data, pos_data, threshold=0.8):
    """
    Align NER tokens to POS tokens using sequence alignment with fuzzy matching.
    Prints unmatched NER tokens.
    """
    ner_words = [t["WORD"] for t in ner_data]
    pos_words = [t["WORD"] for t in pos_data]

    n = len(ner_words)
    m = len(pos_words)

    # Initialize DP table
    dp = [[0]*(m+1) for _ in range(n+1)]
    back = [[None]*(m+1) for _ in range(n+1)]

    # Fill DP table
    for i in range(n+1):
        for j in range(m+1):
            if i > 0 and j > 0:
                score = fuzzy_ratio(ner_words[i-1], pos_words[j-1])
                match_score = dp[i-1][j-1] + (1 if score >= threshold else 0)
                dp[i][j] = match_score
                back[i][j] = (i-1, j-1)
            if i > 0 and (dp[i][j] < dp[i-1][j]):
                dp[i][j] = dp[i-1][j]
                back[i][j] = (i-1, j)
            if j > 0 and (dp[i][j] < dp[i][j-1]):
                dp[i][j] = dp[i][j-1]
                back[i][j] = (i, j-1)

    # Traceback
    i, j = n, m
    aligned_pairs = []
    matched_ner_indices = set()

    while i > 0 and j > 0:
        prev = back[i][j]
        if prev is None:
            break
        pi, pj = prev
        if pi == i-1 and pj == j-1:
            # Diagonal -> matched
            if fuzzy_ratio(ner_words[i-1], pos_words[j-1]) >= threshold:
                aligned_pairs.append((ner_data[i-1], pos_data[j-1]))
                matched_ner_indices.add(i-1)
        # Move to previous cell
        i, j = pi, pj

    aligned_pairs.reverse()  # traceback goes backwards

    # Report unmatched NER tokens
    unmatched_ner = [ner_data[idx]["WORD"] for idx in range(n) if idx not in matched_ner_indices]
    if unmatched_ner:
        print(f"Unmatched NER tokens ({len(unmatched_ner)}):", unmatched_ner)

    # Build DataFrame
    data = []
    for ner_token, pos_token in aligned_pairs:
        combined = {
            # "SENTENCE_ID": pos_token.get("SENTENCE_ID"),
            "WORD": pos_token["WORD"],
            "NER_TAG": ner_token["NER_TAG"],
            "POS_TAG": pos_token["POS_TAG"],
            "LEMMA": pos_token["LEMMA"],
            "FEATS": pos_token["FEATS"],
            "HEAD": pos_token["HEAD"],
            "DEPREL": pos_token["DEPREL"],
            "DEPS": pos_token["DEPS"],
            "MISC": pos_token["MISC"],
        }
        data.append(combined)

    return pd.DataFrame(data)


In [4]:
def split_punct_tokens(tokens):
    """
    Split tokens that contain punctuation attached to words into separate tokens.
    e.g. '08:00' -> ['08', ':', '00'], 'word,' -> ['word', ',']
    Returns a new list of token dicts.
    """
    new_tokens = []
    for t in tokens:
        word = str(t['WORD'])
        # Split into alphanumeric and punctuation sequences
        parts = re.findall(r'\w+|[^\w\s]', word)
        if len(parts) == 1:
            new_tokens.append(t)
        else:
            for p in parts:
                new_t = t.copy()
                new_t['WORD'] = p
                new_tokens.append(new_t)
    return new_tokens

def normalize_for_matching(tokens):
    """Join tokens with spaces for fuzzy matching."""
    text = ' '.join([str(t['WORD']).strip() for t in tokens])
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def strip_punct(text):
    """Remove leading/trailing punctuation for fuzzy comparison."""
    return re.sub(r'^[^\w]+|[^\w]+$', '', text)

def fuzzy_ratio(a, b):
    return SequenceMatcher(None, strip_punct(a), strip_punct(b)).ratio()


In [5]:
def align_ner_to_pos_dp_split(ner_data, pos_data, threshold=0.8, max_span=1):
    """
    Align NER and POS tokens (after splitting punctuation) using DP with fuzzy matching.
    Returns a DataFrame.
    """
    # Split punctuation in both NER and POS
    ner_data_split = split_punct_tokens(ner_data)
    pos_data_split = split_punct_tokens(pos_data)

    n = len(ner_data_split)
    m = len(pos_data_split)

    # DP table and backtracking
    dp = [[0]*(m+1) for _ in range(n+1)]
    back = [[None]*(m+1) for _ in range(n+1)]

    # Fill DP table
    for i in range(n+1):
        for j in range(m+1):
            if i > 0 and dp[i][j] < dp[i-1][j]:
                dp[i][j] = dp[i-1][j]
                back[i][j] = (i-1, j)  # skip NER
            if j > 0 and dp[i][j] < dp[i][j-1]:
                dp[i][j] = dp[i][j-1]
                back[i][j] = (i, j-1)  # skip POS
            if i > 0 and j < m:
                # Try matching NER[i-1] to 1..max_span consecutive POS tokens
                for span in range(1, max_span+1):
                    if j+span > m:
                        break
                    candidate = normalize_for_matching(pos_data_split[j:j+span])
                    ner_word = str(ner_data_split[i-1]['WORD']).strip()
                    score = fuzzy_ratio(ner_word, candidate)
                    if score >= threshold:
                        match_score = dp[i-1][j] + 1
                        if match_score > dp[i][j+span-1]:
                            dp[i][j+span-1] = match_score
                            back[i][j+span-1] = (i-1, j, span)

    # Traceback
    i, j = n, m
    aligned_pairs = []
    unmatched_ner = []

    while i > 0 and j > 0:
        if back[i][j] is None:
            break
        prev = back[i][j]
        if len(prev) == 3:
            pi, pj, span = prev
            pos_tokens = pos_data_split[pj:pj+span]
            aligned_pairs.append((ner_data_split[pi], pos_tokens))
            i, j = pi, pj
        else:
            pi, pj = prev
            if pi == i-1 and pj == j:
                unmatched_ner.append(ner_data_split[i-1]['WORD'])
            i, j = pi, pj

    aligned_pairs.reverse()

    # Build DataFrame
    data = []
    for ner_token, pos_tokens in aligned_pairs:
        combined_word = ''.join([t['WORD'] for t in pos_tokens])
        combined = {
            # "SENTENCE_ID": pos_tokens[-1].get("SENTENCE_ID"),
            "WORD": combined_word,
            "NER_TAG": ner_token["NER_TAG"],
            "POS_TAG": ' '.join([str(t['POS_TAG']) for t in pos_tokens]),
            "LEMMA": ' '.join([str(t['LEMMA']) for t in pos_tokens]),
            "FEATS": ' '.join([str(t['FEATS']) for t in pos_tokens]),
            "HEAD": ' '.join([str(t['HEAD']) for t in pos_tokens]),
            "DEPREL": ' '.join([str(t['DEPREL']) for t in pos_tokens]),
            "DEPS": ' '.join([str(t['DEPS']) for t in pos_tokens]),
            "MISC": ' '.join([str(t['MISC']) for t in pos_tokens]),
        }
        data.append(combined)

    if unmatched_ner:
        print(f"Unmatched NER tokens ({len(unmatched_ner)}):", unmatched_ner)

    return data

In [6]:
def get_ner_data(path):
    ner_words = []
    with Path(path).open(encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            line = line.replace("\t\t'", "\t")
            parts = [normalize_quotes(part.strip()) for part in line.split("\t") if part.strip()]
            if len(parts) >= 2:
                ner = {
                    "WORD": parts[0],
                    "NER_TAG":parts[1]
                }
                ner_words.append(ner)
    return ner_words

def get_pos_data(path):
    pos_words = []

    with Path(path).open(encoding="utf-8") as f:
      data = f.read()

    sentences = parse(data)
    for sentence in sentences:
        for token in sentence:
                pos = {
                    "WORD": normalize_quotes(token["form"]),
                    "LEMMA": normalize_quotes(token["lemma"]),
                    "POS_TAG": token.get("upostag"),
                    "FEATS": token.get("feats"),
                    "HEAD": token.get("head"),
                    "DEPREL": token.get("deprel"),
                    "DEPS": token.get("deps"),
                    "MISC": token.get("misc"),

                }
                pos_words.append(pos)

    return pos_words

In [ ]:
ner_data = get_ner_data(Path("../../Corpus/Files/NER Files in Corpus"))
pos_data = get_pos_data(Path("../../Corpus/Files/POS Files in Corpus"))

In [ ]:

def natural_key(filename):
    """Sort helper: splits string into list of ints and text for natural order sorting."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

def extract_number_and_suffix(filename):
    match = re.match(r"(\\d+)(?:_data.*?headline(\\d+))?", filename)
    if match:
        main_num = int(match.group(1))
        suffix = int(match.group(2)) if match.group(2) else 0
        return (main_num, suffix)
    match = re.match(r"(\\d+)(?:_(\\d+))?\\.conllu", filename)
    if match:
        main_num = int(match.group(1))
        suffix = int(match.group(2)) if match.group(2) else 0
        return (main_num, suffix)
    return (0, 0)

def process_file_pair(ner_file, pos_file):
    ner_data = get_ner_data(ner_file)
    pos_data = get_pos_data(pos_file)
    dataset = align_ner_to_pos_dp_split(ner_data, pos_data)
    print(f"Processed {ner_file.name} and {pos_file.name}")
    return dataset


In [ ]:
def merge_both_datasets(ner_path,pos_path):
    final_dataset = []
    for i in range(1,11):
        ner_subdir = ner_path / f"{i}Part"
        pos_subdir = pos_path / f"{i}Part"
        ner_files = sorted(ner_subdir.glob("*.txt"), key=lambda f: natural_key(f.name))
        pos_files = sorted(pos_subdir.glob("*.conllu"), key=lambda f: natural_key(f.name))
        for ner_file, pos_file in zip(ner_files, pos_files):
            dataset = file_pairs.append((ner_file, pos_file))
            final_dataset.append(dataset)
    return pd.concat(final_dataset, ignore_index=True)

def merge_both_datasets_with_threads(ner_path, pos_path):
    final_dataset = []
    file_pairs = []
        for i in range(1, 11):
            ner_subdir = ner_path / f"6Part"
            pos_subdir = pos_path / f"6Part"
            ner_files = sorted(ner_subdir.glob("*.txt"), key=lambda f: natural_key(f.name))
            pos_files = sorted(pos_subdir.glob("*.conllu"), key=lambda f: natural_key(f.name))
            for ner_file, pos_file in zip(ner_files, pos_files):
                file_pairs.append((ner_file, pos_file))
    max_workers = os.cpu_count() * 4
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_file_pair, ner_file, pos_file) for ner_file, pos_file in file_pairs]
        for future in futures:
            result = future.result()
            if isinstance(result, list):
                result = pd.DataFrame(result)
            final_dataset.append(result)
    return pd.concat(final_dataset, ignore_index=True)

In [ ]:
ner_path = Path("../../Corpus/Files/NER Files in Corpus")
pos_path = Path("../../Corpus/Files/POS Files in Corpus")

dataset = merge_both_datasets(ner_path,pos_path)

In [23]:
dataset.to_csv("output.csv", index=False)